In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from fuzzywuzzy import fuzz
from datetime import date
import datetime as dt
import pandas as pd
import numpy as np
import random
from selenium.webdriver.chrome.options import Options
import re
from tqdm import tqdm
import warnings
import selenium
warnings.filterwarnings("ignore")

C:\Users\Pedro\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
driver = webdriver.Chrome()

In [13]:
start_url = "https://books.toscrape.com/"
driver.get(start_url)

In [14]:
all_book_links = []
page_count = 1

# --- Loop para navegar por todas as páginas (lógica similar) ---
while True:
    print(f"Coletando links da página: {page_count}")

    # Encontra todos os "cards" de livros na página atual
    books = driver.find_elements(By.CLASS_NAME, "product_pod")

    # --- Loop para extrair o link de cada livro na página ---
    for book in books:
        try:
            # >>> MUDANÇA: Encontramos a tag 'a' dentro do 'h3' e pegamos o atributo 'href'
            link_element = book.find_element(By.TAG_NAME, "h3").find_element(By.TAG_NAME, "a")
            link = link_element.get_attribute("href")
            all_book_links.append(link)
        except Exception as e:
            print(f"Erro ao extrair o link de um livro: {e}")
            continue

    # --- Lógica de Paginação (sem alterações) ---
    try:
        # Procura pelo botão "Next"
        next_button = driver.find_element(By.CLASS_NAME, "next")
        next_button.find_element(By.TAG_NAME, "a").click()
        page_count += 1
        time.sleep(3) # Pequena pausa para o carregamento da página
    except NoSuchElementException:
        # Se não encontrar o botão "Next", saímos do loop
        print("Chegamos na última página. Fim da coleta de links.")
        break

Coletando links da página: 1
Coletando links da página: 2
Coletando links da página: 3
Coletando links da página: 4
Coletando links da página: 5
Coletando links da página: 6
Coletando links da página: 7
Coletando links da página: 8
Coletando links da página: 9
Coletando links da página: 10
Coletando links da página: 11
Coletando links da página: 12
Coletando links da página: 13
Coletando links da página: 14
Coletando links da página: 15
Coletando links da página: 16
Coletando links da página: 17
Coletando links da página: 18
Coletando links da página: 19
Coletando links da página: 20
Coletando links da página: 21
Coletando links da página: 22
Coletando links da página: 23
Coletando links da página: 24
Coletando links da página: 25
Coletando links da página: 26
Coletando links da página: 27
Coletando links da página: 28
Coletando links da página: 29
Coletando links da página: 30
Coletando links da página: 31
Coletando links da página: 32
Coletando links da página: 33
Coletando links da 

In [22]:
rating_map = {
    "One": 1.0,
    "Two": 2.0,
    "Three": 3.0,
    "Four": 4.0,
    "Five": 5.0
}


In [28]:
all_books_details = []

for url in tqdm(all_book_links):
    
    driver.get(url)
    time.sleep(2)
    try:
        # Extrai as informações
        title = driver.find_element(By.TAG_NAME, "h1").text
        price = driver.find_element(By.CLASS_NAME, "price_color").text
        try:
            # Remove o '£' e converte para float
            price_float = float(price.replace('£', ''))
        except ValueError:
            price_float = 0.0 # Valor padrão em caso de erro    
            
            
            
        # Disponibilidade: 'In stock (22 available)' -> extrai apenas o texto
        availability = driver.find_element(By.CLASS_NAME, "instock.availability").text
        match = re.search(r'\d+', availability)
        if match:
            availability_int = int(match.group(0))
        else:
            availability_int = 0 # Valor padrão se não encontrar número
            
            
        # Rating: A classe é 'star-rating Five', pegamos a segunda parte 'Five'
        rating_element = driver.find_element(By.CSS_SELECTOR, "p.star-rating")
        rating = rating_element.get_attribute("class").replace("star-rating ", "")
        rating_float = rating_map.get(rating, 0.0)
        
        # Categoria: Usa XPath para encontrar o terceiro item da lista de breadcrumb
        category = driver.find_element(By.XPATH, "//ul[@class='breadcrumb']/li[3]/a").text

        # Imagem: Pega o src e junta com a URL base
        image_relative_url = driver.find_element(By.CSS_SELECTOR, "div.item.active img").get_attribute("src")
        image_url = image_relative_url.replace("../..", "https://books.toscrape.com")


        # Adiciona os dados a uma lista de dicionários
        all_books_details.append({
            "titulo": title,
            "preço": price_float,
            "rating": rating_float,
            "disponibilidade": availability_int,
            "categoria": category,
            "imagem": image_url,
            "url_livro": url
        })

        # Pequena pausa para não sobrecarregar o servidor
        # time.sleep(0.5) 

    except Exception as e:
        print(f"  -> Erro ao extrair dados da URL {url}: {e}")
        continue
    

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [43:49<00:00,  2.63s/it]


In [29]:
df = pd.DataFrame(all_books_details)

In [31]:
df.to_csv("books_details.csv",index=False)

In [32]:
import sqlite3

In [33]:
NOME_DO_BANCO = 'challenge1.sqlite'
NOME_DA_TABELA = 'books_details'

In [35]:
conn = sqlite3.connect(NOME_DO_BANCO)

df.to_sql(NOME_DA_TABELA, conn, if_exists='replace', index=False)

conn.close()